In [84]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn import set_config
set_config(display="diagram")
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.metrics import plot_roc_curve, roc_auc_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import plot_roc_curve, auc,roc_curve,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [85]:
df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/complete_car_data.csv')

In [86]:
df

,Car Name,Mileage,Dealer Name,Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,Open recall,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #
0,2016 Nissan Versa 1.6 S+,"60,001 mi.",Bical Auto Mall,4.7,(746 reviews),"$10,699",5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,Not available,Black,Charcoal,Front-wheel Drive,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CN7AP0GL901453,U3389
1,1989 Mazda RX-7,"90,462 mi.",Brick City Motors,3.0,(85 reviews),"$7,995",3.0,[],None reported,No,...,Not available,Black,Black,Rear-wheel Drive,–,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,26991
2,2011 Chrysler 300 Limited,"129,270 mi.",Dash Auto Gallery Inc,4.8,(81 reviews),"$9,891",4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,Not available,Blue,Black / Light Frost,Rear-wheel Drive,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,2C3CA5CG9BH592453,592453
3,2016 GMC Acadia SLT-1,"146,121 mi.",Dash Auto Gallery Inc,4.8,(81 reviews),"$14,291",4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,At least 1 open recall reported,White,Dark Cashmere,All-wheel Drive,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,1GKKVRKD3GJ292301,292301
4,2006 Lexus RX 330,"129,434 mi.",Big T's Auto Sales LLC,NaN,(5 reviews),"$8,795",4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,Not available,White,Tan,Front-wheel Drive,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,2T2GA31U66C051477,10297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,"130,848 mi.",Motors 46,NaN,NaN,"$7,499",4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,Not available,Sage Green Metallic,Gray,All-wheel Drive,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,69868932
7254,2011 Hyundai Santa Fe GLS,"124,413 mi.",Motors 46,NaN,NaN,"$6,995",4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,At least 1 open recall reported,Mineral Gray,Gray,All-wheel Drive,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,69090043
7255,2018 Kia Rio S,"61,380 mi.",Napoli Nissan,NaN,NaN,"$14,800",5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,Not available,Clear White,Black,Front-wheel Drive,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,X3652
7256,2016 GMC Terrain Denali,"105,678 mi.",Lester Glenn Chevrolet,NaN,NaN,"$14,447",4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,Not available,White Frost Tricoat,Jet Black,All-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,G626081B


In [87]:
# Having the mileage column in float rather than string
df['Mileage'] = df['Mileage'].str.replace(' mi.', '').str.replace(',', '')
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Mileage']

0        60001.0
1        90462.0
2       129270.0
3       146121.0
4       129434.0
          ...   
7253    130848.0
7254    124413.0
7255     61380.0
7256    105678.0
7257    115472.0
Name: Mileage, Length: 7258, dtype: float64

In [88]:
df[['Year', 'Make', 'Model', 'Type']] = df['Car Name'].str.extract(r'(\d{4})\s+([^ ]+)\s+([^ ]+)\s+(.*)')
df

,Car Name,Mileage,Dealer Name,Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Year,Make,Model,Type
0,2016 Nissan Versa 1.6 S+,60001.0,Bical Auto Mall,4.7,(746 reviews),"$10,699",5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CN7AP0GL901453,U3389,2016,Nissan,Versa,1.6 S+
1,1989 Mazda RX-7,90462.0,Brick City Motors,3.0,(85 reviews),"$7,995",3.0,[],None reported,No,...,–,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,26991,1989,Mazda,RX-7,
2,2011 Chrysler 300 Limited,129270.0,Dash Auto Gallery Inc,4.8,(81 reviews),"$9,891",4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,2C3CA5CG9BH592453,592453,2011,Chrysler,300,Limited
3,2016 GMC Acadia SLT-1,146121.0,Dash Auto Gallery Inc,4.8,(81 reviews),"$14,291",4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,1GKKVRKD3GJ292301,292301,2016,GMC,Acadia,SLT-1
4,2006 Lexus RX 330,129434.0,Big T's Auto Sales LLC,NaN,(5 reviews),"$8,795",4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,2T2GA31U66C051477,10297,2006,Lexus,RX,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,NaN,NaN,"$7,499",4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,69868932,2011,Subaru,Forester,2.5X Limited
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,NaN,NaN,"$6,995",4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,69090043,2011,Hyundai,Santa,Fe GLS
7255,2018 Kia Rio S,61380.0,Napoli Nissan,NaN,NaN,"$14,800",5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,X3652,2018,Kia,Rio,S
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,NaN,NaN,"$14,447",4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,G626081B,2016,GMC,Terrain,Denali


In [89]:
df.isna().sum()

Car Name                  0
Mileage                   5
Dealer Name             189
Rating                 4622
Review Count           3215
Price                     0
Car Rating                0
Car Reviews               0
Accidents or damage       0
1-owner vehicle           0
Personal use only         0
Open recall               0
Exterior color            0
Interior color            0
Drivetrain                0
MPG                     195
Fuel type                 8
Transmission              0
Engine                    0
VIN                       0
Stock #                 189
Year                      0
Make                      0
Model                     0
Type                      0
dtype: int64

# Nans:
- Mileage                   5
- Dealer Name             189
- Rating                 4622
- Review Count           3215
- MPG                     195
- Stock #                 189
- Fuel type              8

In [90]:
df = df.rename(columns={"Rating": "Dealer Rating"})

In [91]:
# Replacing Dealer Name with Unkown
df['Dealer Name'] = df['Dealer Name'].fillna('Unkown')

In [92]:
df[df['Review Count'].isna()]

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Year,Make,Model,Type
42,2007 INFINITI G35 Sport,122384.0,Unkown,NaN,NaN,"$14,200",4.7,"['August 30, 2020\nBy alberto from DALLAS\nOwn...",None reported,No,...,NaN,Gasoline,6-Speed M/T,306.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,JNKBV61E57M723481,NaN,2007,INFINITI,G35,Sport
74,2017 Hyundai Santa Fe SE,93416.0,Price Wise Auto Sale,NaN,NaN,"$12,995",4.8,"['January 20, 2023\nBy Eddie from Chicago, IL\...",At least 1 accident or damage reported,Yes,...,18–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic,"3.3L V-6 gasoline direct injection, DOHC, vari...",KM8SM4HF4HU181486,HU181486,2017,Hyundai,Santa,Fe SE
76,2004 Cadillac Escalade Base,127381.0,Carz4Us,NaN,NaN,"$6,995",4.5,"['May 14, 2021\nBy Lefty69 from Hancock, Maryl...",None reported,No,...,13–17\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,6.0L V8 16V MPFI OHV,1GYEK63NX4R117963,117963,2004,Cadillac,Escalade,Base
77,2008 Volvo C70 T5,132242.0,Carz4Us,NaN,NaN,"$7,395",4.7,"['December 14, 2019\nBy Jim from Charlotte, NC...",None reported,Yes,...,18–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,2.5L I5 20V MPFI DOHC Turbo,YV1MC67228J058257,058257,2008,Volvo,C70,T5
90,2011 Mercedes-Benz S-Class 4MATIC,115492.0,Certified Cars of Huntington,NaN,NaN,"$11,595",4.8,"['March 25, 2023\nBy david v from california \...",None reported,No,...,14–21\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic,"5.5L V-8 DOHC, variable valve control, engine ...",WDDNG8GBXBA402523,402523,2011,Mercedes-Benz,S-Class,4MATIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,NaN,NaN,"$7,499",4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,69868932,2011,Subaru,Forester,2.5X Limited
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,NaN,NaN,"$6,995",4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,69090043,2011,Hyundai,Santa,Fe GLS
7255,2018 Kia Rio S,61380.0,Napoli Nissan,NaN,NaN,"$14,800",5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,X3652,2018,Kia,Rio,S
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,NaN,NaN,"$14,447",4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,G626081B,2016,GMC,Terrain,Denali


In [93]:
# All Nans in rating to the mean
rating_mean = df['Dealer Rating'].mean()
df['Dealer Rating'].fillna(rating_mean, inplace=True)

In [94]:
# So we can fill those counts with 0 as well
df['Review Count'] = df['Review Count'].fillna(0)

In [95]:
df.isna().sum()

Car Name                 0
Mileage                  5
Dealer Name              0
Dealer Rating            0
Review Count             0
Price                    0
Car Rating               0
Car Reviews              0
Accidents or damage      0
1-owner vehicle          0
Personal use only        0
Open recall              0
Exterior color           0
Interior color           0
Drivetrain               0
MPG                    195
Fuel type                8
Transmission             0
Engine                   0
VIN                      0
Stock #                189
Year                     0
Make                     0
Model                    0
Type                     0
dtype: int64

In [96]:
# NO need for stock # so get rid of it
df.drop(['Stock #'], axis =1, inplace=True)

In [97]:
# dropping the mileage Nans
df = df.dropna(subset=['Mileage'])

In [98]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Nissan Versa 1.6 S+,60001.0,Bical Auto Mall,4.7000,(746 reviews),"$10,699",5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,Front-wheel Drive,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CN7AP0GL901453,2016,Nissan,Versa,1.6 S+
1,1989 Mazda RX-7,90462.0,Brick City Motors,3.0000,(85 reviews),"$7,995",3.0,[],None reported,No,...,Rear-wheel Drive,–,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,1989,Mazda,RX-7,
2,2011 Chrysler 300 Limited,129270.0,Dash Auto Gallery Inc,4.8000,(81 reviews),"$9,891",4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,Rear-wheel Drive,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,2C3CA5CG9BH592453,2011,Chrysler,300,Limited
3,2016 GMC Acadia SLT-1,146121.0,Dash Auto Gallery Inc,4.8000,(81 reviews),"$14,291",4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,All-wheel Drive,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,1GKKVRKD3GJ292301,2016,GMC,Acadia,SLT-1
4,2006 Lexus RX 330,129434.0,Big T's Auto Sales LLC,3.5272,(5 reviews),"$8,795",4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,Front-wheel Drive,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,2T2GA31U66C051477,2006,Lexus,RX,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,3.5272,0,"$7,499",4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,All-wheel Drive,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,2011,Subaru,Forester,2.5X Limited
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,3.5272,0,"$6,995",4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,All-wheel Drive,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,2011,Hyundai,Santa,Fe GLS
7255,2018 Kia Rio S,61380.0,Napoli Nissan,3.5272,0,"$14,800",5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,Front-wheel Drive,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,2018,Kia,Rio,S
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,3.5272,0,"$14,447",4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,All-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,2016,GMC,Terrain,Denali


In [99]:
# we can fill mpg with the average mpg which is 25.4
df['MPG'] = df['MPG'].fillna(25.4)

In [100]:
# we can fill fuel type with gasoline
df['Fuel type'] = df['Fuel type'].fillna('Gasoline')

In [101]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Nissan Versa 1.6 S+,60001.0,Bical Auto Mall,4.7000,(746 reviews),"$10,699",5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,Front-wheel Drive,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CN7AP0GL901453,2016,Nissan,Versa,1.6 S+
1,1989 Mazda RX-7,90462.0,Brick City Motors,3.0000,(85 reviews),"$7,995",3.0,[],None reported,No,...,Rear-wheel Drive,–,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,1989,Mazda,RX-7,
2,2011 Chrysler 300 Limited,129270.0,Dash Auto Gallery Inc,4.8000,(81 reviews),"$9,891",4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,Rear-wheel Drive,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,2C3CA5CG9BH592453,2011,Chrysler,300,Limited
3,2016 GMC Acadia SLT-1,146121.0,Dash Auto Gallery Inc,4.8000,(81 reviews),"$14,291",4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,All-wheel Drive,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,1GKKVRKD3GJ292301,2016,GMC,Acadia,SLT-1
4,2006 Lexus RX 330,129434.0,Big T's Auto Sales LLC,3.5272,(5 reviews),"$8,795",4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,Front-wheel Drive,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,2T2GA31U66C051477,2006,Lexus,RX,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,3.5272,0,"$7,499",4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,All-wheel Drive,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,2011,Subaru,Forester,2.5X Limited
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,3.5272,0,"$6,995",4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,All-wheel Drive,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,2011,Hyundai,Santa,Fe GLS
7255,2018 Kia Rio S,61380.0,Napoli Nissan,3.5272,0,"$14,800",5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,Front-wheel Drive,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,2018,Kia,Rio,S
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,3.5272,0,"$14,447",4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,All-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,2016,GMC,Terrain,Denali


In [102]:
df.isna().sum()

Car Name               0
Mileage                0
Dealer Name            0
Dealer Rating          0
Review Count           0
Price                  0
Car Rating             0
Car Reviews            0
Accidents or damage    0
1-owner vehicle        0
Personal use only      0
Open recall            0
Exterior color         0
Interior color         0
Drivetrain             0
MPG                    0
Fuel type              0
Transmission           0
Engine                 0
VIN                    0
Year                   0
Make                   0
Model                  0
Type                   0
dtype: int64

# NO more nulls

In [103]:
# column_data_types = df.dtypes

# print(column_data_types)
df.dtypes

Car Name                object
Mileage                float64
Dealer Name             object
Dealer Rating          float64
Review Count            object
Price                   object
Car Rating              object
Car Reviews             object
Accidents or damage     object
1-owner vehicle         object
Personal use only       object
Open recall             object
Exterior color          object
Interior color          object
Drivetrain              object
MPG                     object
Fuel type               object
Transmission            object
Engine                  object
VIN                     object
Year                    object
Make                    object
Model                   object
Type                    object
dtype: object

In [104]:
# Since Price is a string, lets convert to a float

# First remove dollar sign
df['Price'] = df['Price'].str.replace('[^\d.]', '', regex=True)

# Now make it a float
df['Price'] = df['Price'].astype(float)

# Confirm the updated data type
print(df['Price'].dtype)

float64


In [105]:
df['Car Rating'] = pd.to_numeric(df['Car Rating'], errors='coerce')
mean_car_rating = df.loc[df['Car Rating'] != 'No Rating Available', 'Car Rating'].mean()
df['Car Rating'] = df['Car Rating'].replace('No Rating Available', mean_car_rating)
df['Car Rating'] = df['Car Rating'].astype(float)

In [106]:
df['Review Count'] = df['Review Count'].str.extract('(\d+)').astype(float)
df.dtypes

Car Name                object
Mileage                float64
Dealer Name             object
Dealer Rating          float64
Review Count           float64
Price                  float64
Car Rating             float64
Car Reviews             object
Accidents or damage     object
1-owner vehicle         object
Personal use only       object
Open recall             object
Exterior color          object
Interior color          object
Drivetrain              object
MPG                     object
Fuel type               object
Transmission            object
Engine                  object
VIN                     object
Year                    object
Make                    object
Model                   object
Type                    object
dtype: object

In [107]:
df['Open recall']

0                         Not available
1                         Not available
2                         Not available
3       At least 1 open recall reported
4                         Not available
                     ...               
7253                      Not available
7254    At least 1 open recall reported
7255                      Not available
7256                      Not available
7257    At least 1 open recall reported
Name: Open recall, Length: 7253, dtype: object

In [108]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Year,Make,Model,Type
0,2016 Nissan Versa 1.6 S+,60001.0,Bical Auto Mall,4.7000,746.0,10699.0,5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,Front-wheel Drive,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CN7AP0GL901453,2016,Nissan,Versa,1.6 S+
1,1989 Mazda RX-7,90462.0,Brick City Motors,3.0000,85.0,7995.0,3.0,[],None reported,No,...,Rear-wheel Drive,–,Gasoline,Automatic,1.3L Rotary,JM1FC3510K0708787,1989,Mazda,RX-7,
2,2011 Chrysler 300 Limited,129270.0,Dash Auto Gallery Inc,4.8000,81.0,9891.0,4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,Rear-wheel Drive,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,2C3CA5CG9BH592453,2011,Chrysler,300,Limited
3,2016 GMC Acadia SLT-1,146121.0,Dash Auto Gallery Inc,4.8000,81.0,14291.0,4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,All-wheel Drive,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,1GKKVRKD3GJ292301,2016,GMC,Acadia,SLT-1
4,2006 Lexus RX 330,129434.0,Big T's Auto Sales LLC,3.5272,5.0,8795.0,4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,Front-wheel Drive,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,2T2GA31U66C051477,2006,Lexus,RX,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,3.5272,NaN,7499.0,4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,All-wheel Drive,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,JF2SHAEC0BH778959,2011,Subaru,Forester,2.5X Limited
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,3.5272,NaN,6995.0,4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,All-wheel Drive,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,5XYZGDAB1BG027604,2011,Hyundai,Santa,Fe GLS
7255,2018 Kia Rio S,61380.0,Napoli Nissan,3.5272,NaN,14800.0,5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,Front-wheel Drive,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,3KPA25AB5JE049182,2018,Kia,Rio,S
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,3.5272,NaN,14447.0,4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,All-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",2GKFLVE30G6260812,2016,GMC,Terrain,Denali


In [115]:
df

,Car Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Car Rating,Car Reviews,Accidents or damage,1-owner vehicle,...,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,Make,Model,Type,Processed_Reviews
0,2016 Nissan Versa 1.6 S+,60001.0,Bical Auto Mall,4.7000,746.0,10699.0,5.0,"['April 10, 2023\nBy Martin W from Syracuse, N...",At least 1 accident or damage reported,No,...,Charcoal,Front-wheel Drive,31–40\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,Nissan,Versa,1.6 S+,martin w syracuse car prettiest car sure say r...
1,1989 Mazda RX-7,90462.0,Brick City Motors,3.0000,85.0,7995.0,3.0,[],None reported,No,...,Black,Rear-wheel Drive,–,Gasoline,Automatic,1.3L Rotary,Mazda,RX-7,,
2,2011 Chrysler 300 Limited,129270.0,Dash Auto Gallery Inc,4.8000,81.0,9891.0,4.8,"['February 11, 2023\nBy Asa from Redding CA \n...",At least 1 accident or damage reported,No,...,Black / Light Frost,Rear-wheel Drive,18–27\n\n\n\nBased on EPA mileage ratings. Use...,E85 Flex Fuel,5-Speed Automatic,3.6L V6 24V MPFI DOHC Flexible Fuel,Chrysler,300,Limited,asa redding car get chrysler super shopper orl...
3,2016 GMC Acadia SLT-1,146121.0,Dash Auto Gallery Inc,4.8000,81.0,14291.0,4.8,"['January 30, 2023\nBy Paul from Highlands Ra...",None reported,No,...,Dark Cashmere,All-wheel Drive,15–22\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,3.6L V6 24V GDI DOHC,GMC,Acadia,SLT-1,paul highlands ranch car acadia spent lot mone...
4,2006 Lexus RX 330,129434.0,Big T's Auto Sales LLC,3.5272,5.0,8795.0,4.5,"['September 1, 2022\nBy Jeff from Ft. Collins...",None reported,No,...,Tan,Front-wheel Drive,19–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,5-Speed Automatic,3.3L V6 24V MPFI DOHC,Lexus,RX,330,jeff collins car best car ever own owned audi ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,2011 Subaru Forester 2.5X Limited,130848.0,Motors 46,3.5272,NaN,7499.0,4.3,"['October 3, 2022\nBy KD from Fairhope, Alaba...",At least 1 accident or damage reported,No,...,Gray,All-wheel Drive,21–27\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,4-Speed Automatic,2.5L H4 16V MPFI DOHC,Subaru,Forester,2.5X Limited,kd fairhope alabama car subaru love never prob...
7254,2011 Hyundai Santa Fe GLS,124413.0,Motors 46,3.5272,NaN,6995.0,4.5,"['March 10, 2023\nBy notme96 from Spokane, WA\...",None reported,No,...,Gray,All-wheel Drive,20–25\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,2.4L I4 16V MPFI DOHC,Hyundai,Santa,Fe GLS,spokane car buy hyundai santa fe limit awd mil...
7255,2018 Kia Rio S,61380.0,Napoli Nissan,3.5272,NaN,14800.0,5.0,"['April 12, 2023\nBy Brenda Shrader from Kans...",None reported,Yes,...,Black,Front-wheel Drive,28–37\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed Automatic,1.6L I4 16V GDI DOHC,Kia,Rio,S,brenda shrader kansa city car car nightmare bo...
7256,2016 GMC Terrain Denali,105678.0,Lester Glenn Chevrolet,3.5272,NaN,14447.0,4.9,"['May 3, 2021\nBy Traci P from Anadarko\nOwns...",None reported,Yes,...,Jet Black,All-wheel Drive,16–23\n\n\n\nBased on EPA mileage ratings. Use...,Gasoline,6-Speed A/T,"3.6L V-6 gasoline direct injection, DOHC, vari...",GMC,Terrain,Denali,traci p car vehicle vehicle right family inter...
